In [1]:
# import処理は時間が掛かるため事前に終わらせておくと良い
import sys
import os
import time
import socket
import tkinter as tk
# wlan ipv4表示
hostname = socket.gethostname()
print(f'wlan IPv4:\"{socket.gethostbyname(hostname)}\"')

os.chdir(os.path.dirname(os.path.abspath("__file__")))
base = os.path.dirname(os.path.abspath("__file__"))
sys.path.append(f"{base}/recognize_image")
sys.path.append(f"{base}/manage_signal")
sys.path.append(f"{base}/install")
sys.path.append(f"{base}/manage_train")

from treasureAreaDetect import TreasureAreaDetect
from mainTrain import Train
from minifig import Minifig
from mainSignal import MainSignal
from competitionSystem import CompetitionSystem
from parallelProcessing import ParallelProcessing

class InputCommand(ParallelProcessing):
    """
    GUIベースでのコマンド入力を可能にするクラス
    """
    def __init__(self):
        super().__init__()
        self.__cycle = 100
        self.canInput = False
        self.__command = ""

    def execute(self):
        if self.canInput == True:
            self.canInput = False
            self.__command = self.__tkinterInput("コマンドを入力して下さい", width=50)
            if self.__command == "":
                self.__command = "None"
    
    def __tkinterInput(self, prompt="", width=20, always_on_top=True):
        result = [None]  # 結果を保存するためのリスト

        def on_submit():
            result[0] = entry.get()
            root.destroy()

        root = tk.Tk()
        root.title('Input')
        if always_on_top:
            root.attributes('-topmost', True)  # 最前面に表示

        tk.Label(root, text=prompt).pack(padx=10, pady=10)
        entry = tk.Entry(root, width=width)
        entry.pack(padx=10, pady=5)
        entry.bind('<Return>', lambda event=None: on_submit())
        tk.Button(root, text="Submit", command=on_submit).pack(pady=10)
        root.lift()        # ウィンドウを前面に持ってくる
        root.focus_force() # ウィンドウをアクティブにする
        entry.focus_set()  # 入力欄にフォーカスを当てる

        root.mainloop()

        return result[0]

    def getCommand(self):
        command = self.__command
        self.__command = ""
        return command

class Main:
    """
    様々なタスク(スレッドを動作させる)
    ｲﾝｽﾀﾝｽ：競技システム(シングルトンクラス)
    タスク：トレジャーエリアの検知
    タスク：ミニフィグの推測処理
    タスク：IoT列車攻略
    タスク：送受信処理
    タスク：コマンド入力
    """
    def __init__(self):
        self.__competitionSystem = CompetitionSystem()
        # ↓コースの指定
        self.__competitionSystem.setCourse('test')
        self.__competitionSystem.loadData('./IoT.json')
        
        # スレッドの初期化
        self.__inputCommand = InputCommand()
        cameraDevice = 0
        self.__treasureAreaDetect = TreasureAreaDetect(100, cameraDevice)
        self.__train = Train()
        self.__minifig = Minifig(100)
        self.__mainSignal = MainSignal()
        print("[ main ]初期化完了")

    def execute(self):
        """
        メインループ関数
        """
        # スレッドの起動
        self.__treasureAreaDetect.startThread()
        #self.__train.startThread()
        self.__minifig.startThread()
        self.__mainSignal.startThread()
        self.__inputCommand.startThread()

        self.__inputCommand.canInput = True
        while True:
            time.sleep(50/1000)
            command = self.__inputCommand.getCommand()
            if not(command == ""):
                if self.process_command(command) == True:
                    break
                self.__inputCommand.canInput = True


    def process_command(self, command):
        """
        コマンド検索関数(コマンド一覧)
        """
        isEnd = False

        if command in ["e", "exit"]:
            # プログラムの終了処理
            self.terminate()
            isEnd = True
        elif command == "c0":
            self.__treasureAreaDetect.changeCamera(0)
            print("[ main ]0番カメラ切り替え")
        elif command == "c1":
            self.__treasureAreaDetect.changeCamera(1)
            print("[ main ]1番カメラ切り替え")
        elif command== "c2":
            self.__treasureAreaDetect.changeCamera(2)
            print("[ main ]2番カメラ切り替え")
        elif command == "wait":
            time.sleep(1000)
        else:
            print("[ main ]無効なコマンドです")
            
        return isEnd

    def terminate(self):
        """
        プログラム及びスレッドの終了をする。
        """
        self.__treasureAreaDetect.stopThread()
        self.__minifig.stopThread()
        self.__mainSignal.stopThread()
        self.__inputCommand.stopThread()

        del self.__treasureAreaDetect
        del self.__minifig
        del self.__mainSignal
        del self.__competitionSystem 
        del self.__inputCommand
        print("[ main ]プログラムの終了")

wlan IPv4:"10.17.11.102"


In [2]:
### 各種デバイスセッティングができたら押す
    
## 準備期間でやること
# ①画面にカメラデバイスが起動し検知済みかどうかを確認できるので、カメラの向き調整を調整してください
# ②走行体との通信関係ができているかを確認(通信相手のIPアドレスが表示される)

# 一度プログラムを走らせたらカーネルを再起動推奨

# cd /opt/compesys
# sudo npm run start

if __name__ == "__main__":
    main = Main()
    main.execute()

[ 競技システム ]競技システム初回起動
[ 競技システム ] testコース用のipv4 "192.168.100.1"を設定しました。
受信用サーバ起動
送信用サーバ起動
Server started : 0.0.0.0:8080
[ main ]初期化完了
Server started : 0.0.0.0:8081
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リクエストが失敗しました。ステータスコード: 500
[ IoT_GET ]処理データエラー: :各種データの情報がありません
[ IoT_SET ]リク